In [ ]:
from flask import Flask
from flask import request
from flask import jsonify
import dateutil.parser
from datetime import datetime

import cv2
import numpy as np
import pytesseract
from PIL import Image
from pytesseract import image_to_string
import imutils
import os
import base64
pytesseract.pytesseract.tesseract_cmd = 'C:\\Program Files\\Tesseract-OCR\\tesseract.exe'




# import base64
# with open("bill.jpg", "rb") as img_file:
#     my_string = base64.b64encode(img_file.read())
# print(my_string)




def get_string(img_path):
    img_path=os.path.join(img_path)
    img = cv2.imread(img_path,cv2.IMREAD_GRAYSCALE)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    kernel = np.ones((2, 2), np.uint8)
    img=cv2.erode(img,kernel,iterations=1)
    #cv2.imwrite("thres.png", img)
    result = pytesseract.image_to_string(img)
    return result




def extract_date(txt):
    date_expr1 = r"\d{1,2}(?:/|-| )(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec|January|February|March|April|May|June|July|August|September|October|November|December|\d{1,2})(?:/|-| )\d{1,4}"
    date_expr2 = r"(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec|January|February|March|April|May|June|July|August|September|October|November|December)(?:\ |)\d{1,2}(?:\,|')(?:\ |)\d{1,4}"
    date_expr3 = r"\d{1,2}(?: |)(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec|January|February|March|April|May|June|July|August|September|October|November|December)(?:/|')\d{1,4}"

    import re
    if re.search(date_expr1,txt):
        val = re.search(date_expr1,txt).group()
    elif re.search(date_expr2,txt):
        val = re.search(date_expr2,txt).group()
    elif re.search(date_expr3,txt):
        val = re.search(date_expr3,txt).group()
    else:
        val = 'null'
    return val





filename = 'some_image.jpg'

def start_process(my_string):
    imgdata = base64.decodestring(my_string)  
    with open(filename, 'wb') as f:
        f.write(imgdata)

    txt = get_string(os.path.join(filename))

    resultDate=extract_date(txt)
    if resultDate == 'null':
        dictonary = {"date":resultDate}


        print(dictonary)

        return dictonary
    
    newDate= dateutil.parser.parse(resultDate)
    
    datetimeobject=datetime.strptime(str(newDate), '%Y-%m-%d %H:%M:%S')
    
    newDate=datetimeobject.strftime('%Y-%m-%d')
    dictonary = {"date":newDate}


    print(dictonary)

    return dictonary



app = Flask(__name__)

@app.route('/', methods=['POST'])
def get_tasks():
    image_string=request.data
    
    return jsonify(start_process(image_string))

if __name__ == '__main__':
    app.run(debug=True,use_reloader = False)


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
C:\Users\MUJ\Anaconda3\lib\site-packages\ipykernel_launcher.py:64: DeprecationWarning: decodestring() is a deprecated alias since Python 3.1, use decodebytes()
127.0.0.1 - - [27/Nov/2019 22:16:45] "POST / HTTP/1.1" 200 -


{'date': '2017-12-31'}
